# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns = 'loan_status').copy()

# Create our target
y = df['loan_status']

In [10]:
# taking and transpose and displaying all rows just for visibility
X.describe().T.style

,count,mean,std,min,25%,50%,75%,max
loan_amnt,68817.000000,16677.594562,10277.348590,1000.000000,9000.000000,15000.000000,24000.000000,40000.000000
int_rate,68817.000000,0.127718,0.048130,0.060000,0.088100,0.118000,0.155700,0.308400
installment,68817.000000,480.652863,288.062432,30.890000,265.730000,404.560000,648.100000,1676.230000
annual_inc,68817.000000,88213.705210,115579.958439,40.000000,50000.000000,73000.000000,104000.000000,8797500.000000
dti,68817.000000,21.778153,20.199244,0.000000,13.890000,19.760000,26.660000,999.000000
delinq_2yrs,68817.000000,0.217766,0.718367,0.000000,0.000000,0.000000,0.000000,18.000000
inq_last_6mths,68817.000000,0.497697,0.758122,0.000000,0.000000,0.000000,1.000000,5.000000
open_acc,68817.000000,12.587340,6.022869,2.000000,8.000000,11.000000,16.000000,72.000000
pub_rec,68817.000000,0.126030,0.336797,0.000000,0.000000,0.000000,0.000000,4.000000
revol_bal,68817.000000,17604.142828,21835.880400,0.000000,6293.000000,12068.000000,21735.000000,587191.000000


# Encoding the object column "homeowner" and dropping first column

In [11]:
# Create the LabelEncoder instance
X = pd.get_dummies(X)
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [12]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [13]:
# Create X_train, X_test, y_train, y_test
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22, stratify=y)

# Shape X_train
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler()

In [16]:
# Scale the training and testing data
X_train_scaled, X_test_scaled = scaler.transform(X_train), scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Import balanced random forest classifier (BRF)
from imblearn.ensemble import BalancedRandomForestClassifier

In [18]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=22)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=22)

In [21]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.800058216597975

In [22]:
# Display the confusion matrix
cm_brf = pd.DataFrame(confusion_matrix(y_test, y_pred), 
             index = ["High Risk (Predicted Positive)", "Low Risk (Predicted Negative)"], 
             columns = ["Positive", "Negative"])
cm_brf

,Positive,Negative
High Risk (Predicted Positive),63,24
Low Risk (Predicted Negative),2123,14995


In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.72      0.88      0.06      0.80      0.62        87
   low_risk       1.00      0.88      0.72      0.93      0.80      0.64     17118

avg / total       0.99      0.88      0.72      0.93      0.80      0.64     17205



In [25]:
# List the features sorted in descending order by feature importance
feature_importance = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse = True)

[(0.06413407134601781, 'total_pymnt'),
 (0.060662726262810523, 'total_rec_prncp'),
 (0.05681039429132027, 'total_rec_int'),
 (0.053972714670421224, 'total_pymnt_inv'),
 (0.04877198245911794, 'last_pymnt_amnt'),
 (0.027896818043961352, 'int_rate'),
 (0.020694204597187797, 'dti'),
 (0.01859192697851181, 'max_bal_bc'),
 (0.017923961782667017, 'issue_d_Jan-2019'),
 (0.017223652900327492, 'mths_since_recent_inq'),
 (0.01719757742510683, 'out_prncp'),
 (0.017167648030225822, 'annual_inc'),
 (0.01696311723372451, 'tot_hi_cred_lim'),
 (0.016546668621481863, 'avg_cur_bal'),
 (0.01619445851799015, 'mo_sin_old_il_acct'),
 (0.01560315008173812, 'bc_open_to_buy'),
 (0.015530843598627346, 'il_util'),
 (0.015514726470026689, 'revol_bal'),
 (0.01548273863424331, 'mo_sin_old_rev_tl_op'),
 (0.015248586436799525, 'installment'),
 (0.014964851823483092, 'bc_util'),
 (0.014480316644473286, 'mths_since_rcnt_il'),
 (0.014158534967876167, 'total_acc'),
 (0.01367726197522323, 'out_prncp_inv'),
 (0.013345860206

### Easy Ensemble Classifier

In [26]:
# Import the Easy Ensemble Classifier (EEC)
from imblearn.ensemble import EasyEnsembleClassifier

In [27]:
# Train the Classifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=22)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=22)

In [28]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9164088215268461

In [29]:
# Display the confusion matrix
cm_eec = pd.DataFrame(confusion_matrix(y_test, y_pred), 
             index = ["High Risk (Predicted Positive)", "Low RIsk (Predicted Negative)"], 
             columns = ["Positive", "Negative"])
cm_eec

,Positive,Negative
High Risk (Predicted Positive),78,9
Low RIsk (Predicted Negative),1091,16027


In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.90      0.94      0.12      0.92      0.84        87
   low_risk       1.00      0.94      0.90      0.97      0.92      0.84     17118

avg / total       0.99      0.94      0.90      0.96      0.92      0.84     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    - **Easy Ensemble Classifier had the best balanced accuracy score of 0.916**  
    
    
2. Which model had the best recall score?

    - **Easy Ensemble Classifier had the best recall score of 0.94**  
    
    
3. Which model had the best geometric mean score?

    - **Easy Ensemble Classifier had the best geometric mean score of 0.92**  
    
    
4. What are the top three features?

    - **total_pymnt**
    - **total_rec_prncp**
    - **total_rec_int**
 